In [1]:
import paho.mqtt.client as mqtt 
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings
import pickle
from sklearn.preprocessing import StandardScaler
from joblib import dump, load
import pickle

In [8]:
# Load the model
with open('ensemble_model.pkl', 'rb') as f:
    ensemble_model = pickle.load(f)

In [9]:
ensemble_model

VotingClassifier(estimators=[('model_dtree_mpu9050_1',
                              GridSearchCV(cv=5,
                                           estimator=DecisionTreeClassifier(random_state=0),
                                           param_grid={'criterion': ['gini',
                                                                     'entropy'],
                                                       'max_depth': range(1, 10),
                                                       'max_features': ['auto',
                                                                        'sqrt',
                                                                        'log2'],
                                                       'min_samples_leaf': [1,
                                                                            3,
                                                                            5],
                                                       'min_samples_split': [2,
                                                                             4,
                                                                             8]},
                                           scoring='accuracy')),
                             ('model_knn_mpu9050_1',
                              KNeighborsClassifier(n_neighbors=3)),
                             ('model_knn_mpu9050_2',
                              KNeighborsClassifier(n_neighbors=3))])

In [10]:
# Ignore specific warnings of UserWarning type
warnings.filterwarnings("ignore", category=UserWarning)

In [11]:
broker = "127.0.0.1"
port = 1883
topic = "SensorUnoIMU"

def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected to MQTT broker")
        client.subscribe(topic)
    else:
        print("Error connecting, result code: " + str(rc))

def on_message(client, userdata, msg):
    global l_mqtt
    msg = str(msg.payload.decode("utf-8"))    
    dfs = sc.transform(np.asarray([eval(msg)]))
    print("Prediccion: ",ensemble_model.predict(dfs))

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message
client.connect(broker, port, keepalive=60)

client.loop_start()
try:
    while True:
        pass
except KeyboardInterrupt:
    print("Desconectando...")
    client.loop_stop()
    client.disconnect()

C:\Users\biomedica.qm\AppData\Local\Temp\ipykernel_16508\979838774.py:18: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


ConnectionRefusedError: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión